In [1]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
import json

In [2]:
!pip install transformers==4.27.4
!pip install wandb
!pip install datasets
!pip install bert_score
!pip install evaluate
!pip install accelerate
!pip install gradio --upgrade
!pip install --upgrade torch
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 kB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=a0d78a215e9c0a5a9ca7e2495ec0a1e1b9631d5a8d75819330130f6a5cdcd6df
  Stored in directory: /root/.cache/pip/wheels

In [3]:
#text preprocessing
def parse_spoiler_for_correct_spoiler_positions(df):
    spoiler = []        
    for sp in df['spoilerPositions']:
        if len(sp) <=1:
          df['spoilerPositions'] = [[[4, 37],[4, 222]]]  # this data is manually checked and assigned because it was throwing error
        else:
          start,end = sp[0],sp[1]
          spoiler.append(df['targetParagraphs'][start[0]][start[1]:end[1]])
        
    return spoiler

In [4]:
#text preprocessing
def find_token_positions(x):    
    token_position = []
    for position in x['spoilerPositions']:
        if len(position) <2:
          print("format of data is not right for uuid", x['uuid'])
        else:
          index = 0
          start,end = position
          
          for idx,element in enumerate([x['targetTitle']] + x['targetParagraphs']):
              if idx==start[0]+1:
                  # because character is starting at given index +1
                  end_index = index + end[1]
                  start_index = index+start[1]
                  token_position.append([start_index,end_index])
                  break
              if idx==0:
                  index+=len(element)+3
              else:
                  index+=len(element)+1
        
    return token_position

In [5]:
#formatting dataframe columns accroding to requirements
def formatted_df(df):
  
  df['spoilerParsed'] = df.apply(parse_spoiler_for_correct_spoiler_positions,1)
  df['combinedParagraphs'] = df['targetParagraphs'].apply(lambda x:" ".join(x),1)
  df.combinedParagraphs = df.targetTitle + " - " + df.combinedParagraphs
  df['tokenPosition'] = df.apply(find_token_positions,1) 
  df['tags'] = df.tags.apply(lambda x:x[0],1)
  df['postText'] = df.postText.apply(lambda x:x[0],1)
  df['label'] = df['tags'].map({"phrase":0,"passage":1,"multi":2})
  return df
    

In [6]:
data_path = './train.jsonl'
df = pd.read_json(data_path, lines=True)
df_valid = pd.read_json("./validation.jsonl", lines=True)
formatted_df(df)
formatted_df(df_valid)

,uuid,postId,postText,postPlatform,targetParagraphs,targetTitle,targetDescription,targetKeywords,targetMedia,targetUrl,provenance,spoiler,spoilerPositions,tags,spoilerParsed,combinedParagraphs,tokenPosition,label
0,6dc7ddef-4e8e-4a6b-9296-526377518071,800048986762423_885081814925806,Five Nights at Freddy’s Sequel Delayed for Wei...,Facebook,[Five Nights at Freddy’s creator Scott Cawthon...,Five Nights at Freddy’s Sequel Delayed for Wei...,Five Nights at Freddy's creator Scott Cawthon ...,None,[https://gamerant.com/wp-content/uploads/five-...,https://gamerant.com/five-nights-at-freddys-se...,"{'source': 'anonymized', 'humanSpoiler': 'They...",[some of the plot elements are so disturbing t...,"[[[2, 158], [2, 236]]]",passage,[some of the plot elements are so disturbing t...,Five Nights at Freddy’s Sequel Delayed for Wei...,"[[825, 903]]",1
1,435b24de-56f6-4d4e-9c38-54b8e0630aac,4jp20e,Why Arizona Sheriff Joe Arpaio’s fate could ha...,reddit,[© REUTERS/Laura Segall Maricopa County Sherif...,Why Arizona Sheriff Joe Arpaio’s fate could ha...,"<p>A single word — ""intentionally"" — could tra...",None,[https://archive.is/0eGBT/bbf6d79b366fc7e75b8b...,https://archive.is/0eGBT,"{'source': 'anonymized', 'humanSpoiler': '""Int...","[""intentionally"", could transform a court case...","[[[0, 197], [0, 212]], [[0, 215], [0, 328]]]",multi,"[""intentionally"", could transform a court case...",Why Arizona Sheriff Joe Arpaio’s fate could ha...,"[[265, 280], [283, 396]]",2
2,8091ba93-6376-473a-9117-01d6cf0507e4,857559227384160256,Here’s how much you should be tipping your hai...,Twitter,[Here’s how much you should be tipping your ha...,Here’s how much you should be tipping your hai...,Remembering how much you’re supposed to tip an...,,None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",[20%],"[[[3, 58], [3, 61]]]",phrase,[20%],Here’s how much you should be tipping your hai...,"[[729, 732]]",0
3,8b713277-0e5b-4873-a216-b650f21f3b4c,389091583213453312,"""Harry Potter"" alums reunite for new movie",Twitter,[The mythology of punk music's evolution can b...,"Alan Rickman & Rupert Grint On 'CBGB,' Reuniti...",The mythology of punk music's evolution can be...,"Alan Rickman,Hilly Kristal,new rupert grint mo...",[http://s.m.huffpost.com/assets/Logo_Huffingto...,http://huff.to/1ccNwKJ,"{'source': 'anonymized', 'humanSpoiler': 'Alan...","[Alan Rickman & Rupert Grint, CBGB]","[[[-1, 0], [-1, 27]], [[0, 98], [0, 102]]]",multi,"[""CBGB"" is out in limited re, CBGB]","Alan Rickman & Rupert Grint On 'CBGB,' Reuniti...","[[0, 27], [195, 199]]",2
4,a2f91b65-c36c-481f-92b1-2fc77d6411fc,744740593046274048,A man swallowed a microSD card and you won't b...,Twitter,[PetaPixel is one of my favorite blogs. The wr...,Man swallowed a microSD card and you won't bel...,PetaPixel is one of my favorite blogs. The wri...,None,[https://cdn0.vox-cdn.com/uploads/chorus_image...,http://www.theverge.com/circuitbreaker/2016/6/...,"{'source': 'anonymized', 'humanSpoiler': 'This...",[a man who swallowed a 64GB microSD card and t...,"[[[1, 34], [1, 108]]]",passage,[a man who swallowed a 64GB microSD card and t...,Man swallowed a microSD card and you won't bel...,"[[317, 391]]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,1189d343-42eb-47e7-8395-ff978a683875,428006164904034305,This is what happens when you leave a hotel cl...,Twitter,[Instead of encountering a mound of dirty towe...,This Is What Happens When You Leave A Hotel Cl...,Instead of encountering a mound of dirty towel...,"givebackfilms,give back films,video,random act...",[http://s.m.huffpost.com/assets/Logo_Huffingto...,http://huff.to/1ebARdm,"{'source': 'anonymized', 'humanSpoiler': 'She ...",[The video below shows the stunned cleaner ini...,"[[[3, 0], [3, 150]]]",passage,[The video below shows the stunned cleaner ini...,This Is What Happens When You Leave A Hotel Cl...,"[[542, 692]]",1
796,7912282b-137b-4098-875d-8ad9f19354a8,806153730206892032,This Texas GOP elector announces

In [7]:
import evaluate
bleu = evaluate.load("bleu",n=4)
squad_metric = evaluate.load("squad")
meteor = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [9]:
# **PASSAGE Generation Task!**
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import nltk
from nltk import sent_tokenize
from itertools import chain

nltk.download('punkt')
bleu = evaluate.load("bleu")
squad_metric = evaluate.load("squad")
df_para = df_valid[df_valid.tags=="passage"][["uuid","targetTitle","postText","targetParagraphs","spoiler","tokenPosition"]].reset_index(drop=True)
df_para["targetParagraphsWithTitle"] = df_para.apply(lambda x: [x['targetTitle']] + x['targetParagraphs'] if x['targetTitle']!=x['postText'] else x['targetParagraphs'],1)
df_para["answers"] = df_para.apply(lambda x: {'text':x['spoiler'], "answer_start":[x['tokenPosition'][0][0]]},1)
df_para["sents"] = df_para.targetParagraphsWithTitle.apply(lambda x: list(chain.from_iterable([sent_tokenize(y) for y in x])),1)
#name = "cross-encoder/ms-marco-TinyBERT-L-2-v2"
# name = "cross-encoder/stsb-TinyBERT-L-4"
name = "cross-encoder/ms-marco-MiniLM-L-12-v2"

model = AutoModelForSequenceClassification.from_pretrained(name)
tokenizer = AutoTokenizer.from_pretrained(name)

best_paras = []
model.eval()

for ind, row in tqdm(df_para.iterrows(), total=len(df_para)):
    paras = row['targetParagraphs']
    query = [row['postText']]*len(paras)
    features = tokenizer(query,paras,  padding=True, truncation=True, return_tensors="pt")
    
    with torch.no_grad():
        scores = [x.item() for x in model(**features).logits]
    scores = list(zip(scores,paras))
    scores.sort(reverse=True)
    best_paras.append([x[1] for x in scores[:5]])
best_sents = []

for i in tqdm(range(len(best_paras))):
    candidates = []
    for p in best_paras[i][:1]:
        candidates+=sent_tokenize(p)
        
    query  = [df_para.iloc[i]["postText"]]*len(candidates)
    features = tokenizer(query, candidates,  padding=True, truncation=True, return_tensors="pt")
    

    with torch.no_grad():
        scores = [x.item() for x in model(**features).logits]
    scores = list(zip(scores, candidates))
    scores.sort(reverse=True)
    best_sents.append(scores[0][1])
theoretical_texts_squad = [{"id": ex["uuid"], "answers": ex["answers"]} for ind,ex in df_para.iterrows()]
theoretical_texts = [i['answers']['text'][0] for ind,i in df_para.iterrows()] 
predictions = [sent for i,sent in enumerate(best_sents)]

dataframe = pd.DataFrame({'actual_spoiler': theoretical_texts , 'gen_spoiler': predictions })


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  0%|          | 0/322 [00:00<?, ?it/s]

  0%|          | 0/322 [00:00<?, ?it/s]

In [13]:

predictions_squad = [{"id":df_para.iloc[i]["uuid"],"prediction_text":sent} for i,sent in enumerate(best_sents)]

In [14]:
dataframe


,actual_spoiler,gen_spoiler
0,some of the plot elements are so disturbing th...,Cawthon’s reason for suddenly delaying Five Ni...
1,a man who swallowed a 64GB microSD card and th...,He pooped a microSD card!
2,McGonagall was appointed as Dumbledore’s assis...,"From reading J.K. Rowling‘s Harry Potter saga,..."
3,All the scenes are actually in the movie,The filmmaker was directly asked if fans can e...
4,"""I had fake relationships, fake fights. I don'...","Earlier this month, Cavallari opened up to And..."
...,...,...
317,"This was clearly a crack house, and he was sta...",Well that was the driving force behind a man n...
318,China is too quick to rebalance its services s...,"Last week, the US House of Representatives hel..."
319,the plant the dog was chewing on was deadly wa...,"An hour later, the dog was dead."
320,The video below shows the stunned cleaner init...,"Kyle and Josh from GiveBackFilms, a YouTube ch..."


In [15]:
bleu.compute(predictions=best_sents, references=theoretical_texts)


{'bleu': 0.17026280162814836,
 'precisions': [0.2598113427273786,
  0.1572897761645493,
  0.14578874480674808,
  0.14105760398897782],
 'brevity_penalty': 1.0,
 'length_ratio': 1.146155899626268,
 'translation_length': 8587,
 'reference_length': 7492}

In [16]:
squad_metric.compute(predictions=predictions_squad, references=theoretical_texts_squad)


{'exact_match': 6.211180124223603, 'f1': 24.256705284810092}

In [17]:
meteor.compute(predictions=best_sents, references=theoretical_texts)

{'meteor': 0.27176240191972434}